In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime as dt
import requests
import numpy as np
import pandas as pd

In [2]:
artarr = np.zeros((210, 4), dtype='>U60')

In [3]:
def selection(driver, addrow, i, syllable):
    """Move into detailed info of each artists then crawl them"""
    driver.implicitly_wait(10)

    artistClick = driver.find_element_by_css_selector('#pageList > div.list_atist12.d_artist_list > ul > li:nth-child('+ str(i) +') > div > div > dl > dt > a')
    artistClick.click()

    driver.implicitly_wait(10)

    detailinfo = driver.find_element_by_css_selector('#conts > div.wrap_tab_atist > ul > li:nth-child(2) > a')
    detailinfo.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    artist = soup.select('#conts > div.wrap_dtl_atist > div > div.wrap_atist_info > p')
    features_dt = soup.select('#conts > div.section_atistinfo03 > dl > dt')
    features = soup.select('#conts > div.section_atistinfo03 > dl > dd')

    artistproc1 = artist[0].text.strip().replace("아티스트명","")
    artistproc2 = artistproc1.replace(u"\xa0", u" ")
    artarr[5*syllable+i+addrow, 0] = artistproc2

    j = 1
    if features_dt[0].text.strip() == "데뷔":
        if features_dt[2].text.strip() == "결성일":
            for k in [1, 3, 4]:
                feature = features[k].text.strip().split('\n')

                artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                j = j + 1
        else:
            for k in range(1, 4):
                feature = features[k].text.strip().split('\n')
                artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                j = j + 1
    elif features_dt[0].text.strip() == "활동년대":
         for k in range(0, 3):
            feature = features[k].text.strip().split('\n')
            artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
            j = j + 1

    driver.back()
    driver.back()

In [4]:
def korean(driver):
    """Crawl from ㄱ to ㅎ, Numbers and others"""
    for syllable in range(1, 17):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div.wrap_btn_atist.first_child > button:nth-child('+str(syllable)+')')
            click.click()
            #((syllable-1)*5) + i-1
            selection(driver, -6, i, syllable)

In [5]:
def english(driver):
    """Crawl from A to Z"""
    for syllable in range(1, 27):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div:nth-child(2) > button:nth-child('+str(syllable)+') > span > span')
            click.click()
            #((syllable-1)*5) + i-1 + 80
            selection(driver, 74, i, syllable)

In [6]:
def crawlArtistFeatures(url):
    """Crawl top 5 Artists' name, year, type and genre for each syllable"""
    res = requests.get(url, headers={"User-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0 rv:11.0) like Gecko"})
    res.raise_for_status()
    driver = webdriver.Chrome()
    driver.get(url)
    
    korean(driver)
    english(driver)

    driver.quit()

In [7]:
crawlArtistFeatures('https://www.melon.com/artistplus/finder/index.htm')

In [48]:
artarr

array([['기리보이 (홍시영)', '2010 년대', '솔로|남성',
        '발라드, 댄스, 랩/힙합, R&B/Soul, 인디음악'],
       ['김필', '2010 년대', '솔로|남성',
        '발라드, 댄스, R&B/Soul, 인디음악, 록/메탈, 포크/블루스'],
       ['김소유 (김소희)', '2010 년대', '솔로|여성', '성인가요'],
       ['김하온 (HAON)', '2010 년대', '솔로|남성', '발라드, 랩/힙합'],
       ['김동률', '1990, 2000, 2010 년대', '솔로|남성', '발라드'],
       ['나플라(nafla)', '2010 년대', '솔로|남성', '랩/힙합, 인디음악'],
       ['뉴이스트', '2010 년대', '그룹|남성', '발라드, 댄스'],
       ['나얼 (유나얼)', '1990, 2000, 2010 년대', '솔로|남성',
        '발라드, R&B/Soul, CCM, 국내CCM'],
       ['넬 (NELL)', '2000, 2010 년대', '그룹|남성',
        '발라드, 댄스, POP, 록/메탈, 일렉트로니카'],
       ['노을', '2000, 2010 년대', '그룹|남성', '발라드, 댄스, 랩/힙합, R&B/Soul, 록/메탈'],
       ['다비치', '2000, 2010 년대', '그룹|여성', '발라드, 댄스, 랩/힙합, R&B/Soul'],
       ['다이나믹 듀오', '2000, 2010 년대', '그룹|남성',
        '발라드, 댄스, 랩/힙합, R&B/Soul, 록/메탈, CCM, 국내CCM'],
       ['드림캐쳐', '2010 년대', '그룹|여성', '발라드, 댄스, POP, 록/메탈, J-POP'],
       ['디오 (D.O.)', '2010 년대', '솔로|남성', '발라드, R&B/Soul'],
       ['동방신기 (TVXQ!)', 

In [189]:
# Data preprocessing
bdf = pd.DataFrame(artarr, columns=['artists', 'year', 'type', 'genre'])
bdf.set_index('artists', inplace=True)
df = bdf[(bdf['year']!='')]
df.shape
# df.isnull().values.any()

(207, 3)

In [190]:
# Change Korean to English language in DataFrame
pd.options.mode.chained_assignment = None
df['year'] = df['year'].str.replace(' 년대', '')
df['type'] = df['type'].str.replace('|', ', ')
df['type'] = df['type'].str.replace('솔로', 'Solo')
df['type'] = df['type'].str.replace('그룹', 'Group')
df['type'] = df['type'].str.replace('남성', 'Male')
df['type'] = df['type'].str.replace('여성', 'Female')
df['type'] = df['type'].str.replace('혼성', 'Both')
df['genre'] = df['genre'].str.replace('발라드', 'Ballad')
df['genre'] = df['genre'].str.replace('댄스', 'Dance')
df['genre'] = df['genre'].str.replace('랩/힙합', 'Rap/Hiphop')
df['genre'] = df['genre'].str.replace('인디음악', 'Indie')
df['genre'] = df['genre'].str.replace('록/메탈', 'Rock/Metal')
df['genre'] = df['genre'].str.replace('포크/블루스', 'Folk/Blues')
df['genre'] = df['genre'].str.replace('국내CCM', 'KoreanCCM')
df['genre'] = df['genre'].str.replace('뉴에이지', 'Newage')
df['genre'] = df['genre'].str.replace('일렉트로니카', 'Electronica')
df['genre'] = df['genre'].str.replace('포크', 'Folk')
df['genre'] = df['genre'].str.replace('재즈', 'Jazz')
df['genre'] = df['genre'].str.replace('성인가요', 'Trot')
df['genre'] = df['genre'].str.replace('월드뮤직', 'World')
df['genre'] = df['genre'].str.replace('컨트리', 'Country')
df['genre'] = df['genre'].str.replace('블루스', 'Blues')
df['genre'] = df['genre'].str.replace('워십', 'Worship')
df['genre'] = df['genre'].str.replace('클래식', 'Classics')
df['genre'] = df['genre'].str.replace('크로스오버', 'Crossover')
df['genre'] = df['genre'].str.replace('보컬Jazz', 'VocalJazz')
df['genre'] = df['genre'].str.replace('애시드/퓨전/팝', 'Acid/Fusion/Pop')
df['genre'] = df['genre'].str.replace('보사노바', 'BossaNova')
df['genre'] = df['genre'].str.replace('국외ccm', 'ForeignCCM')
df['genre'] = df['genre'].str.replace('만화', 'Comedy')
df['genre'] = df['genre'].str.replace('성악', 'Vocal')
df['genre'] = df['genre'].str.replace('오페라', 'Opera')
df['genre'] = df['genre'].str.replace('중국음악', 'Chinese')
df['genre'] = df['genre'].str.replace('키즈', 'Kids')
df['genre'] = df['genre'].str.replace('\(.*\)', '', regex = True)
df.sample(20)

,year,type,genre
artists,,,
Adele (Adele Laurie Blue Adkins),"2000, 2010","Solo, Female","POP, Rock/Metal, Electronica, R&B/Soul, Classi..."
Taylor Swift (Taylor Alison Swift),"2000, 2010","Solo, Female","POP, Rock/Metal, Electronica, Folk, Country, C..."
첸 (CHEN) (김종대),2010,"Solo, Male","Ballad, Dance, Rap/Hiphop, Folk/Blues, Electro..."
다이나믹 듀오,"2000, 2010","Group, Male","Ballad, Dance, Rap/Hiphop, R&B/Soul, Rock/Meta..."
러블리즈,2010,"Group, Female","Ballad, Dance"
펀치 (Punch),2010,"Solo, Female","Ballad, Rap/Hiphop, R&B/Soul"
E SENS,2010,"Solo, Male","Rap/Hiphop, Indie"
사이먼 도미닉 (정기석),"2000, 2010","Solo, Male","Ballad, Dance, Rap/Hiphop"
Lauv,2010,"Solo, Male","POP, Electronica, EDM"


In [191]:
df.to_csv("artists.csv", encoding='utf-8')

In [216]:
# data_type = {'artists': 'str', 'year': 'str', 'type': 'str', 'genre': 'str'}
df = pd.read_csv("artists.csv", encoding = "utf-8", index_col='artists')
df.shape

(207, 3)

In [217]:
df['year'].astype('str')
df['year'] = df['year'].map(lambda x: x.split(','))
df['type'] = df['type'].map(lambda x: x.split(','))
df['genre'] = df['genre'].map(lambda x: x.split(','))
df.sample(5)

,year,type,genre
artists,,,
Pink Sweat$,[2010],"[Solo, Male]",[R&B/Soul]
Why Don`t We,[2010],"[Group, Male]","[POP, Electronica, EDM]"
DPR LIVE (홍다빈),[2010],"[Solo, Male]","[Indie, Rap/Hiphop]"
GRAY (그레이),[2010],"[Solo, Male]","[Ballad, Rap/Hiphop, R&B/Soul, Indie]"
Lauv,[2010],"[Solo, Male]","[POP, Electronica, EDM]"


In [218]:
df['bagOfWords'] = ''
for index, row in df.iterrows():
    words = ''
    for col in ['year','type','genre']:
        words = words + ' '.join(row[col])+ ' '
    row['bagOfWords'] = words

textdf = df[['bagOfWords']]

In [219]:
textdf.sample(5)

,bagOfWords
artists,
40,2010 Solo Male Ballad Dance Indie Rap/Hiph...
Pink Sweat$,2010 Solo Male R&B/Soul
최서현,2010 Solo Male Rock/Metal Rap/Hiphop
offonoff (오프온오프),2010 Group Male Electronica R&B/Soul
Shawn Mendes,2010 Solo Male POP Rock/Metal Electronica ...


In [220]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# Attempt1: CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(textdf['bagOfWords'])
pd.DataFrame(X.toarray(), columns=vec.get_feature_names()).sample(10)

,04,1950,1960,1970,1980,1990,2000,2001,2009,2010,...,soul,trot,vocal,vocaljazz,world,worship,국외ccm,그룹,남성,빅히트엔터테인먼트
118,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
31,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
168,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
64,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
74,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
162,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
145,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [221]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(X,X)
cosine_sim

array([[1.        , 0.70352647, 0.33333333, ..., 0.60302269, 0.4472136 ,
        0.37796447],
       [0.70352647, 1.        , 0.30151134, ..., 0.36363636, 0.40451992,
        0.34188173],
       [0.33333333, 0.30151134, 1.        , ..., 0.30151134, 0.4472136 ,
        0.56694671],
       ...,
       [0.60302269, 0.36363636, 0.30151134, ..., 1.        , 0.67419986,
        0.68376346],
       [0.4472136 , 0.40451992, 0.4472136 , ..., 0.67419986, 1.        ,
        0.6761234 ],
       [0.37796447, 0.34188173, 0.56694671, ..., 0.68376346, 0.6761234 ,
        1.        ]])

In [222]:
# Attempt2: TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vec2 = TfidfVectorizer()
X2 = vec2.fit_transform(textdf['bagOfWords'])
pd.DataFrame(X2.toarray(), columns=vec2.get_feature_names()).sample(10)

,04,1950,1960,1970,1980,1990,2000,2001,2009,2010,...,soul,trot,vocal,vocaljazz,world,worship,국외ccm,그룹,남성,빅히트엔터테인먼트
40,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.123798,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.208265,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.0,0.000000,0.517468,0.0,0.0,0.239198,...,0.384710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.251153,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.684492,0.456081,0.0,0.0,0.210822,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.000000,0.585181,0.0,0.0,0.270499,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.000000,0.276865,0.0,0.0,0.127980,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.377708,0.0,0.0,0.174595,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.000000,0.358419,0.0,0.0,0.165678,...,0.266465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.000000,0.503173,0.0,0.0,0.232590,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
cosine_sim2 = cosine_similarity(X2,X2)
cosine_sim2

array([[1.        , 0.57090708, 0.10697939, ..., 0.36132804, 0.26107302,
        0.20842399],
       [0.57090708, 1.        , 0.07649232, ..., 0.15464792, 0.18667223,
        0.14902716],
       [0.10697939, 0.07649232, 1.        , ..., 0.07698644, 0.14482319,
        0.27164463],
       ...,
       [0.36132804, 0.15464792, 0.07698644, ..., 1.        , 0.53158917,
        0.65029973],
       [0.26107302, 0.18667223, 0.14482319, ..., 0.53158917, 1.        ,
        0.58292019],
       [0.20842399, 0.14902716, 0.27164463, ..., 0.65029973, 0.58292019,
        1.        ]])

In [224]:
indices = pd.Series(textdf.index)
indices

0                             기리보이 (홍시영)
1                                     김필
2                              김소유 (김소희)
3                             김하온 (HAON)
4                                    김동률
                     ...                
202                       ZENE THE ZILLA
203                         Zion.T (김해솔)
204               Zedd (Anton Zaslavski)
205                    ZAYN (Zayn Malik)
206    Zara Larsson (Zara Maria Larsson)
Name: artists, Length: 207, dtype: object

In [237]:
def recommendations(artist, matrix):
    
    top10 = []
    sims = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == artist].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(matrix[idx]).sort_values(ascending = False)
#     print(score_series)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        top10.append(list(df.index)[i])
        
    return top10

In [240]:
recommendations('기리보이 (홍시영)', cosine_sim)

['40',
 'Crush (신효섭)',
 'GRAY (그레이)',
 'Ugly Duck (선주경)',
 'BewhY (비와이)',
 '매드클라운',
 '로꼬 (권혁우)',
 '헤이즈 (Heize)',
 '창모 (CHANGMO)',
 'pH-1 (박준원)']

In [241]:
recommendations('기리보이 (홍시영)', cosine_sim2)

['40',
 'BewhY (비와이)',
 'Ugly Duck (선주경)',
 'GRAY (그레이)',
 'Crush (신효섭)',
 '창모 (CHANGMO)',
 'pH-1 (박준원)',
 'Hash Swan',
 '매드클라운',
 '헤이즈 (Heize)']